In [1]:
import torch
import torch.nn as nn                 
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from model import *
# Define the Vision Transformer model

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

from tqdm import tqdm

import pandas as pd
# auto reload
%load_ext autoreload
%autoreload 2


In [2]:

    
# Define the Vision Transformer model
class VisionTransformer(nn.Module):
    def __init__(self, num_classes, embed_dim, dim, num_heads, img_size, patch_size, in_channels=3):
        super(VisionTransformer, self).__init__()
        self.num_patches = (img_size // patch_size) ** 2
        self.patch_embedding = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.positional_embedding = nn.Parameter(torch.randn(1, self.num_patches, embed_dim))
        self.hopfield = KHopfield(N=dim, n=embed_dim * self.num_patches)
        self.fc = nn.Linear(embed_dim * self.num_patches, num_classes)
        self.num_heads = num_heads

    def forward(self, x):
        x1 = self.patch_embedding(x)  # (batch_size, embed_dim, num_patches_h, num_patches_w)
        x2 = x1.permute(0, 2, 3, 1)  # (batch_size, num_patches_h, num_patches_w, embed_dim)
        x3 = x2.reshape(x2.size(0), -1, x2.size(-1))  # (batch_size, num_patches, embed_dim)
        
        x4 = x3 + self.positional_embedding  # Add positional embedding
        # combine second and third dimension
        x5 = x4.flatten(1, 2)
        x6 = self.hopfield(x5, self.num_heads)
        x7 = x6.mean(dim=2)  # Global average pooling
        x8 = self.fc(x7)
        return x8
    
    def to(self, device):
        super(VisionTransformer, self).to(device)
        self.hopfield = self.hopfield.to(device)
        return self
    
# Uses #num_heads k=1 hopfield networks, rather than k=num_heads-hopfield networks
class VisionTransformerV(nn.Module):
    def __init__(self, num_classes, embed_dim, dim, num_heads, img_size, patch_size, in_channels=3):
        super(VisionTransformerV, self).__init__()
        self.num_patches = (img_size // patch_size) ** 2
        self.patch_embedding = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.positional_embedding = nn.Parameter(torch.randn(1, self.num_patches, embed_dim))

        self.hopfields = nn.ModuleList([KHopfield(N=dim, n=embed_dim * self.num_patches) for _ in range(num_heads)])
        self.fc = nn.Linear(embed_dim * self.num_patches, num_classes)
        self.num_heads = num_heads

    def forward(self, x):
        x1 = self.patch_embedding(x)  # (batch_size, embed_dim, num_patches_h, num_patches_w)
        x2 = x1.permute(0, 2, 3, 1)  # (batch_size, num_patches_h, num_patches_w, embed_dim)
        x3 = x2.reshape(x2.size(0), -1, x2.size(-1))  # (batch_size, num_patches, embed_dim)
        
        x4 = x3 + self.positional_embedding  # Add positional embedding
        # combine second and third dimension
        x5 = x4.flatten(1, 2)
        x6 = [self.hopfields[i](x5, 1) for i in range(self.num_heads)]
        # take average of all heads
        x6 = torch.stack(x6, dim=2).squeeze()
        x7 = x6.mean(dim=2)  # Global average pooling
        x8 = self.fc(x7)
        return x8
    


In [3]:
# # Define the Vision Transformer model
# class VisionTransformer(nn.Module):
#     def __init__(self, num_classes, embed_dim, num_heads, num_layers, img_size, patch_size):
#         super(VisionTransformer, self).__init__()
#         num_patches = (img_size // patch_size) ** 2
#         self.patch_embedding = nn.Conv2d(3, embed_dim, kernel_size=patch_size, stride=patch_size)
#         self.positional_embedding = nn.Parameter(torch.randn(1, num_patches + 1, embed_dim))
#         self.transformer = nn.Transformer(
#             d_model=embed_dim,
#             nhead=num_heads,
#             num_encoder_layers=num_layers,
#             dim_feedforward=2048,
#             dropout=0.1,
#         )
#         self.fc = nn.Linear(embed_dim, num_classes)

#     def forward(self, x):
#         x = self.patch_embedding(x)  # (batch_size, embed_dim, num_patches_h, num_patches_w)
#         x = x.permute(0, 2, 3, 1)  # (batch_size, num_patches_h, num_patches_w, embed_dim)
#         x = x.reshape(x.size(0), -1, x.size(-1))  # (batch_size, num_patches, embed_dim)
#         x = torch.cat([self.positional_embedding, x], dim=1)
#         x = self.transformer(x)
#         x = x.mean(dim=1)  # Global average pooling
#         x = self.fc(x)
#         return x


In [5]:
# Hyperparameters
batch_size = 256
num_epochs = 10
learning_rate = 1e-3
num_classes = 10
img_size = 32  # Assuming CIFAR-10 image size

#mnist

patch_size = 16  # Adjust this based on your preference

datast = 'mnist'
dataset = 'cifar'
num_heads = 4

if dataset == 'mnist':
    transform = transforms.Compose([transforms.ToTensor()])
    train_dataset = torchvision.datasets.MNIST(root='~/data', train=True, transform=transforms.ToTensor(), download=True)
    test_dataset = torchvision.datasets.MNIST(root='~/data', train=False, transform=transforms.ToTensor(), download=True)
    img_size = 28
    in_channels = 1
    num_classes = 10
else:
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    train_dataset = torchvision.datasets.CIFAR10(root='~/data', train=True, transform=transform, download=True)
    test_dataset = torchvision.datasets.CIFAR10(root='~/data', train=False, transform=transform, download=True)
    img_size = 32
    in_channels = 3
    num_classes = 10

# Data preprocessing
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Initialize the model and optimizer
model = VisionTransformer(
    num_classes = num_classes, 
    embed_dim = 256, 
    dim  = 1024,
    num_heads = num_heads, 
    img_size = img_size, 
    patch_size = patch_size, 
    in_channels = in_channels,
)
modelV= VisionTransformerV(
    num_classes = num_classes, 
    embed_dim = 256, 
    dim  = 1024,
    num_heads = num_heads, 
    img_size = img_size, 
    patch_size = patch_size, 
    in_channels = in_channels,
)
modelS = SimpleViT(
    image_size = img_size,
    patch_size = patch_size,
    num_classes = num_classes,
    dim = 1024,
    depth = 1,
    heads = 4,
    mlp_dim = 2048
)


optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
modelV = modelV.to(device)
print(device)






Files already downloaded and verified
Files already downloaded and verified
transformer 1
cuda:3


In [3]:
def validate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return 100 * correct / total

In [7]:
# Training loop
for epoch in range(num_epochs):

    # show loss in tqdm
    model.train()
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)

    
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        pbar.set_description(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {loss.item():.4f}')

    # validate
    accuracy = validate(model, test_loader)

    print(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {loss.item():.4f}, Val Accuracy: {accuracy:.4f}')
    

print('Training finished!')

  0%|          | 0/196 [00:00<?, ?it/s]

Epoch [1/10] Loss: 2.2988:   1%|          | 2/196 [00:00<00:29,  6.64it/s]

shapes
torch.Size([256, 1024, 4])
torch.Size([256, 1024])
shapes
torch.Size([256, 1024, 4])
torch.Size([256, 1024])


Epoch [1/10] Loss: 2.2893:   2%|▏         | 4/196 [00:00<00:28,  6.76it/s]

shapes
torch.Size([256, 1024, 4])
torch.Size([256, 1024])
shapes
torch.Size([256, 1024, 4])
torch.Size([256, 1024])


...

Epoch [1/10] Loss: 2.0463:  16%|█▋        | 32/196 [00:04<00:21,  7.75it/s]

shapes
torch.Size([256, 1024, 4])
torch.Size([256, 1024])
shapes
torch.Size([256, 1024, 4])
torch.Size([256, 1024])


Epoch [1/10] Loss: 1.9906:  17%|█▋        | 33/196 [00:04<00:21,  7.45it/s]


shapes
torch.Size([256, 1024, 4])
torch.Size([256, 1024])


KeyboardInterrupt: 

In [8]:
X = torch.nn.Parameter(torch.randn(59, 16))

hopfield = KHopfield(N=100, n=16 )

# optimize the hopfield network
optimizer = optim.Adam(hopfield.parameters(), lr=learning_rate)

for i in range(10):
    print(i)
    Y = hopfield(X, 4)
    loss = torch.norm(Y)
    loss.backward()
    print(loss)
    optimizer.step()

0
tensor(0.8274, grad_fn=<CopyBackwards>)
1
tensor(0.7943, grad_fn=<CopyBackwards>)
2
tensor(0.7643, grad_fn=<CopyBackwards>)
3
tensor(0.7368, grad_fn=<CopyBackwards>)
4
tensor(0.7115, grad_fn=<CopyBackwards>)
5
tensor(0.6887, grad_fn=<CopyBackwards>)
6
tensor(0.6683, grad_fn=<CopyBackwards>)
7
tensor(0.6504, grad_fn=<CopyBackwards>)
8
tensor(0.6348, grad_fn=<CopyBackwards>)
9
tensor(0.6215, grad_fn=<CopyBackwards>)


In [33]:
# Initialize the model and optimizer
num_heads = 3


model = VisionTransformer(
    num_classes = num_classes, 
    embed_dim = 1024, 
    num_heads = num_heads, 
    img_size = img_size, 
    patch_size = patch_size, 
    in_channels = in_channels,
)
model = SimpleViT(
    image_size = img_size,
    patch_size = patch_size,
    num_classes = num_classes,
    dim = 1024,
    depth = 1,
    heads = 4,
    mlp_dim = 2048
)

transformer 1


In [4]:
def get_model_and_data(
        data = 'mnist',
        model = 'hopfield',
        batch_size = 256,
        heads = 4,
        dim=256,
        embed_dim=1024):
    if data == 'mnist':
        transform = transforms.Compose([transforms.ToTensor()])
        train_dataset = torchvision.datasets.MNIST(root='~/data', train=True, transform=transforms.ToTensor(), download=True)
        test_dataset = torchvision.datasets.MNIST(root='~/data', train=False, transform=transforms.ToTensor(), download=True)
        img_size = 28
        in_channels = 1
        num_classes = 10

    elif data  == 'cifar10':
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

        train_dataset = torchvision.datasets.CIFAR10(root='~/data', train=True, transform=transform, download=True)
        test_dataset = torchvision.datasets.CIFAR10(root='~/data', train=False, transform=transform, download=True)
        img_size = 32
        in_channels = 3
        num_classes = 10
    else:
        raise Exception('data not found')
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    patch_size = 16

    if model == 'hopfield':
        model = VisionTransformer(
            num_classes = num_classes, 
            embed_dim = embed_dim, 
            dim = dim,
            num_heads = heads, 
            img_size = img_size, 
            patch_size = patch_size, 
            in_channels = in_channels,
        )
    elif model == 'hopfieldV':
        model = VisionTransformerV(
            num_classes = num_classes, 
            embed_dim = embed_dim, 
            dim = dim,
            num_heads = heads, 
            img_size = img_size, 
            patch_size = patch_size, 
            in_channels = in_channels,
        )
    elif model == 'vit':
        model = SimpleViT(
            image_size = img_size,
            patch_size = patch_size,
            num_classes = num_classes,
            dim = dim,
            depth = 1,
            heads = heads,
            mlp_dim = 1024
        )
    return model, train_loader, test_loader

In [5]:
def run_experiment(train_loader, test_loader, model, num_epochs=30, k=1):
    # Initialize the model and optimizer
    model = model.to(device)
    learning_rate = 1e-3
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    val_accuracy = []
    # Training loop
    for epoch in range(num_epochs):

        # show loss in tqdm
        model.train()
        pbar = tqdm(enumerate(train_loader), total=len(train_loader))
        for i, (images, labels) in pbar:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            pbar.set_description(f'K: {k}, Epoch [{epoch + 1}/{num_epochs}] Loss: {loss.item():.4f}')

        # validate
        accuracy = validate(model, test_loader)

        print(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {loss.item():.4f}, Val Accuracy: {accuracy:.4f}')
        val_accuracy.append(accuracy)

    print('Training finished!')

    return val_accuracy

In [11]:
model, train_loader, test_loader = get_model_and_data(data = 'cifar10', model = 'hopfieldV', batch_size = 256, heads = 4, dim=256)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
val_accuracy  = run_experiment(train_loader, test_loader, model)

  0%|          | 0/196 [00:00<?, ?it/s]

Epoch [1/10] Loss: 1.7703:  69%|██████▉   | 135/196 [00:28<00:12,  4.78it/s]

LML Warning: Did not converge.
LML Warning: Did not converge.
LML Warning: Did not converge.
LML Warning: Did not converge.


Epoch [1/10] Loss: 1.7718:  81%|████████  | 158/196 [00:33<00:07,  4.80it/s]

LML Warning: Did not converge.
LML Warning: Did not converge.
LML Warning: Did not converge.
LML Warning: Did not converge.


...

LML Warning: Did not converge.
LML Warning: Did not converge.
LML Warning: Did not converge.
LML Warning: Did not converge.


Epoch [2/10] Loss: 1.6674:  16%|█▌        | 31/196 [00:14<01:16,  2.15it/s]

LML Warning: Did not converge.
LML Warning: Did not converge.
LML Warning: Did not converge.
LML Warning: Did not converge.


Epoch [2/10] Loss: 1.7895:  17%|█▋        | 34/196 [00:15<01:13,  2.22it/s]


KeyboardInterrupt: 

In [20]:
ks = [1, 4, 8, 16]

df = pd.DataFrame(columns = ['k', 'accuracy', 'model', 'epoch'])
data = 'cifar10'
model = 'hopfield'
for k in ks:
    model, train_loader, test_loader = get_model_and_data(data = data, model = model, batch_size = 256, heads = k, dim=100)
    val_accuracy  = run_experiment(train_loader, test_loader, model, num_epochs=30, k =k )
    # add every val accuracy to dataframe
    for i, acc in enumerate(val_accuracy):
        df = df.append({'k': k, 'accuracy': acc, 'model': 'hopfield', 'epoch': i}, ignore_index=True)
    
    # save dataframe
    df.to_csv(f'./results/{data}_{model}_heads.csv')

Files already downloaded and verified
Files already downloaded and verified


K: 1, Epoch [1/30] Loss: 1.9899:  76%|███████▌  | 148/196 [00:12<00:04, 10.00it/s]

LML Warning: Did not converge.


K: 1, Epoch [1/30] Loss: 1.8530:  93%|█████████▎| 182/196 [00:16<00:01,  9.84it/s]

LML Warning: Did not converge.


...

K: 1, Epoch [3/30] Loss: 1.7803:  54%|█████▍    | 106/196 [00:25<00:21,  4.26it/s]

LML Warning: Did not converge.


K: 1, Epoch [3/30] Loss: 1.8212:  55%|█████▍    | 107/196 [00:25<00:20,  4.27it/s]

LML Warning: Did not converge.


K: 1, Epoch [3/30] Loss: 1.7398:  55%|█████▌    | 108/196 [00:25<00:20,  4.27it/s]

In [15]:
# get df rows
df = pd.read_csv(f'./results/{data}_{model}_heads.csv')

In [19]:
df.iloc[0].item()

ValueError: can only convert an array of size 1 to a Python scalar